In [10]:
import pandas as pd
import random

In [24]:
df_vehicles = pd.read_excel('../Datos_P1/df_vehicle.xlsx')
df_distances_km = pd.read_excel('../Datos_P1/df_distance_km.xlsx')
df_orders = pd.read_excel('../Datos_P1/df_orders.xlsx')

df_location = pd.read_excel('../Datos_P1/df_location.xlsx')

In [ ]:
# PRUEBAS TONTAS
df_distances_km.index = df_distances_km.columns

distance = df_distances_km.at["Cliente_1", "Cliente_2"]
print(distance)

7.5625


In [40]:
def create_initial_population(population_size=10):
    population = []
    
    orders = df_orders.to_dict(orient="records")
    vehicles = df_vehicles.to_dict(orient="records")
    almacen = "Almacén"
    
    for _ in range(population_size):
        individual = []
        remaining_orders = orders.copy()  # Clonar los pedidos restantes
        
        for vehicle in vehicles:
            route = [almacen]  # Cada ruta comienza en el almacén
            capacity_left = vehicle["capacidad_kg"]
            autonomy_left = vehicle["autonomia_km"]
            
            while remaining_orders:
                order = random.choice(remaining_orders)

                if order["order_demand"] <= capacity_left:
                    current_client = route[len(route) - 1]
                    next_client = order["cliente"]
                    distance = df_distances_km.at[current_client, next_client]

                    if distance <= autonomy_left and distance != 0:
                        route.append(next_client)
                        capacity_left -= order["order_demand"]
                        autonomy_left -= distance
                        remaining_orders.remove(order)
                    else:
                        break
                else:
                    break
            
            route.append(almacen)
            individual.append(route)
        
        population.append(individual)
    
    return population

In [68]:
# Generar la población inicial
population = create_initial_population(10)
for individual in population:
    for route in individual:
        print(f'Ruta: {route}')

Ruta: ['Almacén', 'Cliente_19', 'Cliente_9', 'Almacén']
Ruta: ['Almacén', 'Cliente_5', 'Cliente_15', 'Cliente_1', 'Cliente_17', 'Almacén']
Ruta: ['Almacén', 'Cliente_4', 'Cliente_6', 'Cliente_20', 'Cliente_16', 'Cliente_18', 'Almacén']
Ruta: ['Almacén', 'Cliente_3', 'Cliente_10', 'Cliente_2', 'Almacén']
Ruta: ['Almacén', 'Cliente_11', 'Cliente_13', 'Cliente_8', 'Cliente_14', 'Cliente_12', 'Cliente_7', 'Almacén']
Ruta: ['Almacén', 'Almacén']
Ruta: ['Almacén', 'Cliente_1', 'Cliente_5', 'Almacén']
Ruta: ['Almacén', 'Cliente_15', 'Cliente_9', 'Cliente_17', 'Cliente_16', 'Almacén']
Ruta: ['Almacén', 'Cliente_10', 'Cliente_3', 'Cliente_14', 'Cliente_4', 'Cliente_11', 'Almacén']
Ruta: ['Almacén', 'Cliente_6', 'Cliente_2', 'Cliente_7', 'Almacén']
Ruta: ['Almacén', 'Cliente_20', 'Cliente_19', 'Cliente_12', 'Cliente_8', 'Cliente_13', 'Cliente_18', 'Almacén']
Ruta: ['Almacén', 'Almacén']
Ruta: ['Almacén', 'Cliente_10', 'Cliente_1', 'Almacén']
Ruta: ['Almacén', 'Cliente_18', 'Cliente_3', 'Cliente_

In [66]:
def evaluate_population(population):
    fitness_scores = []
    
    # Crear un diccionario de los pedidos por cliente para una búsqueda más eficiente
    orders_dict = df_orders.set_index('cliente')['order_demand'].to_dict()

    for individual in population:
        total_distance = 0
        total_penalty = 0  # Penalización si hay algún error (capacidad, autonomía)
        
        for route, vehicle in zip(individual, df_vehicles.to_dict(orient="records")):
            route_distance = 0
            capacity_left = vehicle["capacidad_kg"]
            autonomy_left = vehicle["autonomia_km"]
            
            # Verifica la distancia y la capacidad/autonomía en cada ruta
            current_location = "Almacén"
            
            for client in route[1:-1]:  # Saltar el almacén al inicio y final
                distance = df_distances_km.at[current_location, client]
                route_distance += distance
                autonomy_left -= distance  # Reducir autonomía con la distancia recorrida
                current_location = client
                
                # Obtener la demanda del pedido del cliente actual de la lista de pedidos
                order_demand = orders_dict.get(client, 0)  # Si no hay pedido, asignamos 0

                # Verificar si la autonomía o capacidad se exceden
                if autonomy_left < 0:
                    total_penalty += 1000  # Penalización por exceder la autonomía
                
                capacity_left -= order_demand  # Reducir la capacidad disponible
                if capacity_left < 0:
                    total_penalty += 1000  # Penalización por exceder la capacidad

            # Añadir la distancia de regreso al almacén
            route_distance += df_distances_km.at[current_location, "Almacén"]
            total_distance += route_distance
        
        # Al final de la ruta, guardamos la aptitud de esta solución
        fitness_scores.append(total_distance + total_penalty)
    
    return fitness_scores


In [71]:
fitness_scores = evaluate_population(population=population)
fitness_scores

[np.float64(274.2727),
 np.float64(235.916),
 np.float64(253.9374),
 np.float64(244.96629999999996),
 np.float64(224.7081),
 np.float64(250.4787),
 np.float64(262.0616),
 np.float64(302.22029999999995),
 np.float64(210.64849999999996),
 np.float64(254.5)]